# Plot brain profiles

In [1]:
import os
os.chdir('/Users/ltran/Documents/TrueData29/CPC_ML_tutorial/')

import numpy as np
import pandas as pd
import pickle
from matplotlib import pyplot as plt
import seaborn as sns
import random

from scipy.stats import fisher_exact
from pcntoolkit.normative import estimate, predict, evaluate
from pcntoolkit.util.utils import compute_MSLL, create_design_matrix
from nm_utils import calibration_descriptives, remove_bad_subjects, load_2d
from sklearn.model_selection import train_test_split
# Set data directory
data_dir = '/Users/ltran/Documents/Data/'

# Set working directory
root_dir = '/Users/ltran/Documents/TrueData0104/CPC_ML_tutorial/'
out_dir = os.path.join(root_dir,'models','test')

# create the output directory if it does not already exist
os.makedirs(out_dir, exist_ok=True)

from enigmatoolbox.utils.parcellation import parcel_to_surface
from enigmatoolbox.plotting import plot_cortical

## Extract Z-Scores

In [3]:
# Function to read Z-scores stored in files

def fetch_raw_Z():
    # Read outputs files 
    Z_pred = {}

    for idp_num, idp in enumerate(idp_ids): 
        idp_dir = os.path.join(out_dir, idp)
        os.chdir(idp_dir)
        f = open("Z_predict.txt", "r")
        Z_pred.update({f'{idp}_Z_predict' : f.readlines()})

    Z_df_pred = pd.DataFrame(Z_pred).astype(float)
    
    return Z_df_pred

In [4]:
# Load train and test sets

df_te3 = pd.read_csv(os.path.join(data_dir, 'Outputs/df_te3.csv'))

idp_ids = [col for col in df_te3.columns if '_thickness' in col]

len(idp_ids)

## Extract Z-Scores

Z_df_pred = fetch_raw_Z()

## Merge Z-Scores with infos

Z_df = Z_df_pred.join(df_te3)

cols_df = [col for col in Z_df.columns if 'Z_predict' in col]

## Define functions

In [23]:
def brain_maps(X):
    CT_d_fsa5 = parcel_to_surface(X, 'aparc_fsa5')
    plot_cortical(array_name=CT_d_fsa5, surface_name="fsa5", size=(800, 400),
                  cmap='RdBu_r', color_bar=True, color_range=(-abs(X).max(), abs(X).max()))

In [24]:
def compute_abs_prevalence(Z_df_est):
    prev_est= {}
    for i in Z_df_est.columns:
        prev_est.update({i : [Z_df_est.query(f'{i} > 2 or {i} < -2').count()[1]/Z_df_est.shape[0]]})
    prev_est = pd.DataFrame.from_dict(prev_est)
    
    return prev_est

def compute_positive_prevalence(Z_df_est):
    prev_est= {}
    for i in Z_df_est.columns:
        prev_est.update({i : [Z_df_est.query(f'{i} > 2').count()[1]/Z_df_est.shape[0]]})
    prev_est = pd.DataFrame.from_dict(prev_est)
    
    return prev_est

def compute_negative_prevalence(Z_df_est):
    prev_est= {}
    for i in Z_df_est.columns:
        prev_est.update({i : [Z_df_est.query(f'{i} < -2').count()[1]/Z_df_est.shape[0]]})
    prev_est = pd.DataFrame.from_dict(prev_est)
    
    return prev_est


## Compute prevalences

In [25]:
Z_df_est = Z_df.loc[Z_df.ASD == 'No'][cols_df]
Z_df_pred = Z_df.loc[Z_df.ASD == 'Yes'][cols_df]

In [26]:
print('TD count : ', Z_df_est.shape[0])
print('ASD count : ', Z_df_pred.shape[0])

TD count :  246
ASD count :  1097


In [27]:
# Compute prevalence of Z > 2 
pos_prev_TD = compute_positive_prevalence(Z_df_est)
pos_prev_ASD = compute_positive_prevalence(Z_df_pred)

In [28]:
# Compute prevalence of |Z| > 2
abs_TD = compute_abs_prevalence(Z_df_est)
abs_ASD = compute_abs_prevalence(Z_df_pred)

In [29]:
# Compute prevalence of Z < -2
neg_prev_TD = compute_negative_prevalence(Z_df_est)
neg_prev_ASD = compute_negative_prevalence(Z_df_pred)

In [30]:
pr_bool = input('To map Z > 2 : "pos"\nTo map Z < -2 : "neg"\nTo map |Z| > 2 : "abs"\n')

To map Z > 2 : "pos"
To map Z < -2 : "neg"
To map |Z| > 2 : "abs"
abs


In [32]:
# Compute pr(|Z|asd > 2) - pr(|Z|td > 2)
if (pr_bool == 'abs'):    
    diff = abs_ASD.values - abs_TD.values
elif (pr_bool == 'pos'):
    diff = pos_prev_ASD.values - pos_prev_TD.values
elif (pr_bool == 'neg'):
    diff = neg_prev_ASD.values - neg_prev_TD.values

In [33]:
brain_maps(diff)